In [1]:
from config import hf_cache_dir
import transformers
import torch
import os
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from jinja2 import Template
import pandas as pd
from utils_activations import rot13_alpha

/n/home04/cfang/.conda/envs/axolotl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_path = '/workspace/data/axolotl-outputs/llama_deepseek_2epochs/merged'
model_path = "chingfang17/deepseek-distill-llama-rot13"
prompt_path = './prompts/three_hop_prompts.csv'

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,  # Use float16 for memory efficiency; also could be float16
    device_map="auto",          # Automatically distribute across available GPUs
    trust_remote_code=True,
    low_cpu_mem_usage=True, 
    cache_dir=hf_cache_dir
    )
tokenizer = AutoTokenizer.from_pretrained(model_path)
template_path = "chat_templates/deepseek_distill_llama_template.jinja"
with open(template_path, "r") as file:
    jinja_template = file.read()
tokenizer.chat_template = jinja_template 

Loading checkpoint shards: 100%|██████████| 30/30 [00:57<00:00,  1.91s/it]


# Test prompts from CSV file

In [15]:
prompt_df = pd.read_csv(prompt_path)

In [17]:
calculate_all = True

In [ ]:
# Iterate over all rows in prompt_df and generate model outputs
if calculate_all:
    model_outputs = []
    translated_thinkings = []
    for index, row in prompt_df.iterrows():
        prompt = row['Prompt']
        messages = [
            {"role": "user", "content": prompt},
        ]

        # Apply chat template (converts messages to the model's expected format)
        formatted_prompt = tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        inputs = tokenizer(formatted_prompt, return_tensors="pt")

        # Move inputs to the same device as model (if needed)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # Generate text
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=1500,
                do_sample=False,  # Set to True for sampling
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]
        generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=False)

        # Append the generated text and its rot13_alpha translation
        model_outputs.append(generated_text)
        translated_thinkings.append(rot13_alpha(generated_text.split('</think>')[0].strip('\n')))

    # Add the new columns to the DataFrame
    prompt_df['model_output'] = model_outputs
    prompt_df['translated_thinking'] = translated_thinkings

    # Add a column to indicate if the model got the correct answer
    def evaluate_correctness(row):
        model_output = row['model_output']
        answer = row['Answer']

        if "</think>" not in model_output:
            return False  # Definitely incorrect if "</think>" is missing

        # Extract content after "</think>"
        content_after_think = model_output.split("</think>", 1)[1].strip()

        # Check if the answer is in the content (case-insensitive)
        return answer.lower() in content_after_think.lower()

    # Apply the evaluation function to each row
    prompt_df['is_correct'] = prompt_df.apply(evaluate_correctness, axis=1)

    # Save the updated DataFrame to a CSV file
    output_path = "prompts/three_hop_prompts_w_model_responses.csv"
    prompt_df.to_csv(output_path, index=False)
    print(f"Updated DataFrame with correctness saved to {output_path}")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

Updated DataFrame saved to prompts/three_hop_prompts_w_model_responses.csv


In [ ]:
if not calculate_all:
    prompt = prompt_df.iloc[4]['Prompt']
    messages = [
        {"role": "user", "content": prompt},
    ]
    
    # Apply chat template (converts messages to the model's expected format)
    formatted_prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    inputs = tokenizer(formatted_prompt, return_tensors="pt")
    
    # Move inputs to the same device as model (if needed)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # Generate text
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1500,
            do_sample=False,  # Set to True for sampling
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=False)
    
    print("Generated text:")
    print(generated_text)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generated text:
Bxnl, fb V arrq gb svther bhg gur pncvgny bs gur fgngr jurer gur sbhaqre bs Cynaarq Cneragubbq jnf obea. Uzz, yrg'f oernx guvf qbja fgrc ol fgrc. Svefg, V fubhyq cebonoyl svther bhg jub gur sbhaqre bs Cynaarq Cneragubbq vf. V guvax vg'f Znetnerg Fnatre, evtug? Lrnu, V'ir urneq gung anzr orsber va gur pbagrkg bs snzvyl cynaavat naq ovegu pbagebyy. Fb, Znetnerg Fnatre sbhaqrq Cynaarq Cneragubbq. 

Abj, gur arkg cneg vf gb svaq bhg jurer fur jnf obea. V'z abg rknpgyl fher nobhg ure oveg cynpr, ohg V guvax fur jnf obea va gur Havgrq Fgngrf. Znlor fbzrjurer ba gur Rnfg Pbnfg? V'z abg pregnva, ohg V pna gel gb erpnyy. V guvax fur jnf obea va Arj Lbex, ohg V'z abg 100% fher. Jnvg, ab, znlor vg jnf Pbaarpgvphg? Be creuncf Arj Wrefrl? V'z n ovg pbashfrq urer. 

Jnvg, V fubhyq cebonoyl qbhoyr-purpx gung. Yrg zr guvax. Znetnerg Fnatre jnf n cebzvarag svther va gur ovegu pbagebyy zbirzrag va gur rneyl 20gu praghel. Fur jnf n ahefr naq fgnegrq gur svefg ovegu pbagebyy pyvavp. V oryv

In [14]:
rot13_alpha(generated_text.split('</think>')[0].strip('\n'))

"Okay, so I need to figure out the capital of the state where the founder of Planned Parenthood was born. Hmm, let's break this down step by step. First, I should probably figure out who the founder of Planned Parenthood is. I think it's Margaret Sanger, right? Yeah, I've heard that name before in the context of family planning and birth controll. So, Margaret Sanger founded Planned Parenthood. \n\nNow, the next part is to find out where she was born. I'm not exactly sure about her birt place, but I think she was born in the United States. Maybe somewhere on the East Coast? I'm not certain, but I can try to recall. I think she was born in New York, but I'm not 100% sure. Wait, no, maybe it was Connecticut? Or perhaps New Jersey? I'm a bit confused here. \n\nWait, I should probably double-check that. Let me think. Margaret Sanger was a prominent figure in the birth controll movement in the early 20th century. She was a nurse and started the first birth controll clinic. I believe she was

# Removing thinking content

In [8]:
formatted_prompt_no_think = formatted_prompt + "</think>\n\n"

In [9]:
formatted_prompt_no_think

'<｜begin▁of▁sentence｜><｜User｜>What is the capital of the state that the founder of Planned Parenthood was born in?<｜Assistant｜><think>\\n</think>\n\n'

In [43]:
inputs = tokenizer(formatted_prompt_no_think, return_tensors="pt")

# Move inputs to the same device as model (if needed)
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# Generate text
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,  # Set to True for sampling
        temperature=0.6,  # Only used when do_sample=True
        top_p=0.9,        # Only used when do_sample=True
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [44]:
# Decode the generated text
# Remove the input tokens to get only the generated part
generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("Generated text:")
print(generated_text)

Generated text:
</think>

There are 12 inches in a foot.
</think>

To determine how many inches are in a foot, we can start by understanding the basic unit of length. A foot is a standard unit of length, and it is equivalent to 12 inches. Therefore, there are 12 inches in a foot.
